In [ ]:
!pip install ydata_profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.8/357.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.5 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=e4d40a35ac8981204d84a4ee73b79a00eece4265d501ba85034b73832f7b0b27
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting unin

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ydata_profiling import ProfileReport

from google.colab import auth
from google.colab import drive
auth.authenticate_user()
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/Colab Notebooks/prj2/'

In [ ]:
df_low_1 = pd.read_csv(path+'bottom 5% order and products.csv')
df_low_2 = pd.read_csv(path+'filtered_orders_low.csv')
df_high_1 = pd.read_csv(path+'top 5% order and products.csv')
df_high_2 = pd.read_csv(path+'filtered_orders_high.csv')

In [ ]:
df3=pd.read_csv(path+'aisles.csv')

## high

In [ ]:
merged_high = pd.concat([df_high_1, df_high_2], ignore_index=True)
merged_high = pd.concat([merged_high, df3], ignore_index=True)

In [ ]:
reorder_rate_high = merged_high.groupby('aisle_id')['reordered'].mean().reset_index()
reorder_rate_high

## low

In [ ]:
merged_low = pd.concat([df_low_1, df_low_2], ignore_index=True)
merged_low = pd.concat([merged_low, df3], ignore_index=True)

In [ ]:
reorder_rate_low = merged_low.groupby('aisle_id')['reordered'].mean().reset_index()
reorder_rate_low

In [ ]:
result = reorder_rate_high['reordered'] - reorder_rate_low['reordered']
result_df = pd.DataFrame({'aisle_id': reorder_rate_high['aisle_id'], 'reordered_difference': result})
result_df

In [ ]:
result_df = pd.merge(result_df, df3, left_on='aisle_id', right_on='aisle_id', how='left')
result_df = result_df.sort_values(by = 'reordered_difference', ascending=False)
result_df

In [ ]:
Profile =  ProfileReport(result_df, title="Profiling Report")
Profile.to_file(path + 'reorder diff.html')

In [ ]:
# Given data
data = {
    'reordered_difference': [0.506231, 0.490805, 0.462380, 0.440852, 0.427736, 0.406076, 0.385413, 0.385280, 0.383441, 0.362409],
    'aisle': ['baby accessories', 'white wines', 'spirits', 'specialty wines champagnes', 'beers coolers',
              'red wines', 'prepared soups salads', 'diapers wipes', 'fresh herbs', 'poultry counter']
}

df = pd.DataFrame(data)

# Sorting the DataFrame based on 'reordered_difference' column
df = df.sort_values(by='reordered_difference', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
plt.barh(df['aisle'], df['reordered_difference'], color='skyblue')
plt.xlabel('Reordered Difference')
plt.title('Top Aisles by Reordered Difference')
plt.show()


## 가설에 대한 데이터 뽑아내기 - 아이 용품을 살 때 주류도 구매한다

In [ ]:
merged_df = pd.merge(df_high_1, df_high_2[['order_id', 'user_id']], on='order_id', how='left')
merged_df

In [ ]:
# 첫 aisle_id의 집합과 다른 집합 정의
baby_aisles = {82, 56, 92, 102, 100}
drink_aisles = {62, 124, 134, 27, 28}

filtered_df = merged_df[df_high_1['aisle_id'].isin(baby_aisles)]
# 추출된 order_id를 사용하여 해당하는 row들 추출
filtered_df = merged_df[merged_df['order_id'].isin(filtered_df['order_id'])]
filtered_df

In [ ]:
unique_user_ids = filtered_df['user_id'].unique()
filtered_df = merged_df[merged_df['user_id'].isin(unique_user_ids)]
filtered_df

In [ ]:
filtered_df2 = filtered_df[filtered_df['aisle_id'].isin(drink_aisles)]
filtered_df2 = df_high_1[df_high_1['order_id'].isin(filtered_df2['order_id'])]
filtered_df2

In [ ]:
unique_order_id_count = filtered_df2['order_id'].nunique()
print("Unique order_id count:", unique_order_id_count)
unique_order_id_count = filtered_df['order_id'].nunique()
print("Unique order_id count:", unique_order_id_count)

## 가설은 기각. 아이용품과 같이 많이 구매하는 상품에 대해서 살펴보자

In [ ]:
# 'aisle_id'를 기준으로 그룹화하고 각 그룹의 row 개수 출력
grouped_df = filtered_df.groupby('aisle_id').size().reset_index(name='row_count')
sorted_grouped_df = grouped_df.sort_values(by='row_count', ascending=False)
# 결과 확인
sorted_grouped_df

In [ ]:
sorted_grouped_df.to_csv(path + 'last.csv')

In [ ]:
import matplotlib.pyplot as plt

# Given data
aisles = ["fresh fruits", "baby food formula", "fresh vegetables", "packaged vegetables fruits",
          "yogurt", "missing", "milk", "packaged cheese", "water seltzer sparkling water",
          "chips pretzels", "bread"]

frequency = [10.58944892, 10.1688535, 9.473169849, 4.976925165, 4.308026423,
              2.922088499, 2.647724188, 2.610804452, 2.083793322, 1.675142521, 1.668265315]

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.barh(aisles, frequency, color='skyblue')
plt.xlabel('Percentage')
plt.title('what top 5% users buy with baby products')
plt.gca().invert_yaxis()  # Invert y-axis for better visualization
plt.show()